## 数据处理

In [1]:
import pandas as pd
import io
import re
from datetime import datetime, timedelta
from typing import List, Union, Optional
from datetime import datetime

import os, re, json, time
import typing as T
import requests

from pathlib import Path
from openpyxl import Workbook, load_workbook

from openpyxl.utils import get_column_letter
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font, Alignment, Border, Side, PatternFill
from data_processing import load_and_process,build_jsonl_for_range, save_jsonl


In [2]:
## 研发字典
speaker_map = {
    "16186514":   "peter本尊",
    "1655611808": "运营绾绾",
    "2073820674": "沙利文老师",
    "2726067525": "milissa",
}
## 客服字典
MAPPING_FILE = "mapping地球1.xlsx"

##QQ的txt文件
pathtxt   = "1209《欢迎来到地球》测试2群.txt"

# 设定时间范围
start_time = "2025-12-07 00:00:00"
end_time   = "2025-12-08 00:00:00"


# 1) 拿到 JSONL（列表）
jsonl_lines01 = build_jsonl_for_range(
    pathtxt=pathtxt,
    mapping_file=MAPPING_FILE,
    speaker_map=speaker_map,
    start_time=start_time,
    end_time=end_time,
    return_str=False,   # 返回 list[str]
)

print(len(jsonl_lines01), "lines")
print(jsonl_lines01[0])



1201 lines
{"发言日期": "2025-12-07", "发言时间": "00:09:54", "玩家ID": "“徐凤年”(2780439644)", "玩家消息": "开车"}


## 大模型分类

## 定义

In [3]:
"""
批处理 10000 条聊天数据（每批 100 条）：
- 模型#1：过滤非游戏相关（只保留相关 JSON 行，原样输出）
- 模型#2：提取高讨论的发言并分析
- 结果按word格式文档输出
"""
from model_classifyV1_Copy1_Copy1 import (
    load_system_prompt,
    build_user_prompt_filter,build_user_prompt_clsuter,call_ark_chat_completions,
    extract_valid_json_lines,add_index_to_jsonl_lines,count_output_filter_stats,get_covered_indices_from_cluster_output,
    aggregate_cluster_outputs,build_user_prompt_cluster_agg,assign_global_cluster_ids,
    extract_top5_heat_clusters,attach_discussion_points,extract_cluster_stats,append_daily_top5_to_version_jsonl,infer_date_for_batch,
    match_dialogs_by_time,build_user_prompt_subcluster_opinion,extract_time_axis_from_title,
    parse_and_normalize_opinion_output,build_daily_top5_opinion_records,build_user_prompt_version_agg,parse_model2_output_to_json_list,normalize_model3_clusters,
    read_jsonl_file,build_version_agg_input_jsonl_text,compute_version_heat_topk)

## 设置参数

In [4]:
# ============= 你的模型与文件配置（改这里） =============
API_URL   = "https://ark.cn-beijing.volces.com/api/v3/chat/completions" 
API_KEY = "de91deb0-aae6-46cb-bac0-17ac3b6107f5" #API
V3_MODEL_ID= "ep-20251020160142-5d7hp"#接入点
V3_1_MODEL_ID = "ep-20251020160025-9p5tj"#接入点
R1_MODEL_ID = "ep-20251020160103-5n6g2"#接入点

PROMPT_MD_PATH01 = Path("提示词1.md") # 模型#1 system 提示词（筛相关）
PROMPT_MD_PATH02 = Path("2话题分类.md") # 模型#2 system 提示词（分话题）
PROMPT_MD_PATH03 = Path("3日聚合.md") # 模型#3 system 提示词（日聚合）
PROMPT_MD_PATH04 = Path("版本聚合.md") #模型#4 system 提示词（观点分析）
VERSION_TOP5_JSONL = "test_top5.jsonl"
BATCH_SIZE       = 300
SLEEP_BETWEEN    = 1   # 每批之间的间隔，防止QPS触发限流；按需调整
RETRIES          = 2
TEMPERATURE      = 0.20
MAX_TOKENS       = 16384
TIMEOUT_SEC      = 600
# =====================================================

## 1.检验每日话提簇分类输出

In [15]:
import json
import time
from tqdm import tqdm



# ===========================================================
# =============== 主循环：模型#1 + 模型#2 ====================
# ===========================================================

# 读取系统提示
system_prompt01 = load_system_prompt(PROMPT_MD_PATH01)  # 筛相关
system_prompt02 = load_system_prompt(PROMPT_MD_PATH02)  # 做话题簇
# 1⃣ 给原始 jsonl 每条加 _idx
jsonl_lines01_indexed = add_index_to_jsonl_lines(jsonl_lines01)

total = len(jsonl_lines01_indexed)

PRINT_UNCLUSTERED = False
if total == 0:
    print("没有可处理的数据。")
else:
    total_batches = (total + BATCH_SIZE - 1) // BATCH_SIZE
    print(f"准备处理 {total} 条，共 {total_batches} 批（每批 {BATCH_SIZE} 条）。")

# 统计总量用的（可选）
total_filtered_lines = 0         # 模型#1 输出总行数之和（含客服）
total_filtered_player_lines = 0  # 模型#1 输出玩家行数之和
total_covered_idx = 0            # 被话题簇覆盖的原始发言总数（按 _idx 去重）


for b in tqdm(range(total_batches), desc="🔥 批处理进度", unit="批"):
    start = b * BATCH_SIZE
    end = min(start + BATCH_SIZE, total)
    # 本批原始发言（已带 _idx）
    batch_lines = jsonl_lines01_indexed[start:end]

    # 本批原始 _idx 集合，用来和话题簇覆盖做差集
    batch_original_idx = set()
    for line in batch_lines:
        try:
            obj = json.loads(line)
            batch_original_idx.add(int(obj["_idx"]))
        except Exception:
            continue

    try:
        # ========== 模型 #1：筛相关 ==========
        user_prompt1 = build_user_prompt_filter(batch_lines)
        output_filter = call_ark_chat_completions(
            api_url=API_URL,
            api_key=API_KEY,
            model=V3_MODEL_ID,
            system_prompt=system_prompt01,
            user_prompt=user_prompt1,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            timeout=TIMEOUT_SEC,
            retries=RETRIES,
        )
        if not output_filter:
            tqdm.write(f"[批次 {b+1}] 模型#1 无有效输出，跳过。")
            continue

        # 统计筛相关之后的行数：总行数 & 玩家行数
        filter_total_lines, filter_player_lines = count_output_filter_stats(output_filter)
        total_filtered_lines += filter_total_lines
        total_filtered_player_lines += filter_player_lines

        tqdm.write(
            f"[批次 {b+1}] 模型#1 输出总行数：{filter_total_lines}，"
            f"其中玩家发言：{filter_player_lines} 条（不去重）"
        )

        # ========== 模型 #2：话题簇 ==========
        user_prompt2 = build_user_prompt_clsuter(output_filter)
        output_cluster = call_ark_chat_completions(
            api_url=API_URL,
            api_key=API_KEY,
            model=V3_1_MODEL_ID,
            system_prompt=system_prompt02,
            user_prompt=user_prompt2,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            timeout=TIMEOUT_SEC,
            retries=RETRIES,
        )
        if not output_cluster:
            tqdm.write(f"[批次 {b+1}] 模型#2 无有效输出，跳过。")
            continue

        # 打印本批的话题簇输出（原样）
        print(f"\n===== 批次 {b+1}/{total_batches} 话题簇输出 =====")
        print(output_cluster)
        print("===========================================\n")

        # 计算：本批有哪些 _idx 被话题簇覆盖（从“发言行号列表”解析）
        covered_idx = get_covered_indices_from_cluster_output(output_cluster)
        total_covered_idx += len(covered_idx)

        # 和“本批原始发言 _idx 集合”做差集，看有哪些没被覆盖
        unclustered_idx = batch_original_idx - covered_idx

        tqdm.write(
            f"[批次 {b+1}] 话题簇覆盖原始发言条数：{len(covered_idx)}，"
            f"未被覆盖条数：{len(unclustered_idx)}"
        )

        # ⭐⭐ 新增：打印“未被话题簇覆盖”的发言信息（最多前 20 条）
        if PRINT_UNCLUSTERED and unclustered_idx:
            # 先把这一批中未覆盖的发言对象捞出来
            unclustered_records = []
            for line in batch_lines:
                try:
                    obj = json.loads(line)
                except Exception:
                    continue
                idx_val = obj.get("_idx")
                try:
                    idx_int = int(idx_val)
                except Exception:
                    continue
                if idx_int in unclustered_idx:
                    unclustered_records.append(obj)

            # 按 _idx 排序，避免乱序
            unclustered_records.sort(key=lambda x: int(x.get("_idx", 0)))

            print(f"\n[批次 {b+1}] 未被话题簇覆盖的原始发言（最多前20条）：")
            for rec in unclustered_records[:20]:
                idx = rec.get("_idx")
                date = rec.get("发言日期") or rec.get("日期") or ""
                t = rec.get("发言时间") or rec.get("时间") or ""
                speaker = rec.get("发言人ID") or rec.get("玩家ID") or rec.get("角色ID") or ""
                msg = (
                    rec.get("玩家消息")
                    or rec.get("发言内容")
                    or rec.get("玩家发言")
                    or rec.get("消息")
                    or ""
                )
                print(f"- _idx={idx} [{date} {t}] {speaker}: {msg}")
            print()  # 换行分隔一下

    except Exception as e:
        tqdm.write(f"[批次 {b+1}] ❌ 出错：{e}")
        continue

    # 防止 QPS 过高
    time.sleep(SLEEP_BETWEEN)

print("\n✅ 全部批次处理完成！")
print(f"原始输入总数：{total}")
print(f"模型#1 筛相关后总行数：{total_filtered_lines}，其中玩家发言：{total_filtered_player_lines} 条")
print(f"模型#2 话题簇累计覆盖原始发言条数：{total_covered_idx}")

准备处理 141 条，共 1 批（每批 300 条）。


🔥 批处理进度:   0%|                                                                                                                                     | 0/1 [02:39<?, ?批/s]      

[批次 1] 模型#1 输出总行数：89，其中玩家发言：89 条（不去重）


🔥 批处理进度:   0%|                                                                                                                                     | 0/1 [02:46<?, ?批/s]      


===== 批次 1/1 话题簇输出 =====
{"话题簇":"停服维护补偿讨论（2025-11-19 15:02:03-15:11:27）","核心对象/机制":"停服维护补偿机制，玩家讨论维护时长与补偿期望（100抽、50抽、石币、代币等）"}
{"话题簇":"游戏下载与安装问题（2025-11-19 15:02:22-15:13:20）","核心对象/机制":"游戏安装包下载与更新机制，玩家询问下载链接、是否需要重新下载等问题"}
{"话题簇":"角色技能更新讨论（2025-11-19 15:02:58-15:04:02）","核心对象/机制":"猛犸象和仙人掌角色的PTSD与强迫症被动技能更新实装情况"}
{"话题簇":"客服互动与调侃（2025-11-19 15:07:27-15:14:50）","核心对象/机制":"客服服务体验，玩家与客服薏米、蓝桉的互动调侃及无关游戏内容的闲聊"}

[批次 1] 话题簇覆盖原始发言条数：0，未被覆盖条数：141


🔥 批处理进度: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [02:47<00:00, 167.86s/批]


✅ 全部批次处理完成！
原始输入总数：141
模型#1 筛相关后总行数：89，其中玩家发言：89 条
模型#2 话题簇累计覆盖原始发言条数：0


## 加讨论观点分析的版本测试

In [ ]:
from tqdm import tqdm
import time
import json
from pathlib import Path

# --- 前置：系统提示、初始化等 ---
system_prompt01 = load_system_prompt(PROMPT_MD_PATH01)  # 筛相关
system_prompt02 = load_system_prompt(PROMPT_MD_PATH02)  # 做话提簇
system_prompt03 = load_system_prompt(PROMPT_MD_PATH03)  # 日话题簇聚合


batch_cluster_outputs = []  # 用于存放当天所有批次的话题簇 JSON 行（给模型#3用）

total = len(jsonl_lines01)
if total == 0:
    print("没有可处理的数据。")
else:
    total_batches = (total + BATCH_SIZE - 1) // BATCH_SIZE
    print(f"准备处理 {total} 条，共 {total_batches} 批（每批 {BATCH_SIZE} 条）。")

written_total = 0

# ====================================================
# 模型 #1 + 模型 #2：筛相关 + 话题簇划分（按批处理）
# ====================================================
for b in tqdm(range(total_batches), desc="🔥 批处理进度", unit="批"):
    start = b * BATCH_SIZE
    end = min(start + BATCH_SIZE, total)
    batch_lines = jsonl_lines01[start:end]

    try:
        # --- 模型 #1：筛相关 ---
        user_prompt1 = build_user_prompt_filter(batch_lines)
        output_filter = call_ark_chat_completions(
            api_url=API_URL,
            api_key=API_KEY,
            model=V3_MODEL_ID,
            system_prompt=system_prompt01,
            user_prompt=user_prompt1,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            timeout=TIMEOUT_SEC,
            retries=RETRIES,
        )
        
        if not output_filter:
            tqdm.write(f"[批次 {b+1}] 模型#1 无有效输出，跳过。")
            continue

        filter_count = sum(1 for line in output_filter.splitlines() if line.strip())
        tqdm.write(f"[批次 {b+1}] 模型#1 筛后保留 {filter_count} 条")
        written_total += filter_count

        # --- 模型 #2：话题簇划分 ---
        user_prompt2 = build_user_prompt_clsuter(output_filter)
        output_cluster = call_ark_chat_completions(
            api_url=API_URL,
            api_key=API_KEY,
            model=V3_1_MODEL_ID,
            system_prompt=system_prompt02,
            user_prompt=user_prompt2,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            timeout=TIMEOUT_SEC,
            retries=RETRIES,
        )

        if not output_cluster:
            tqdm.write(f"[批次 {b+1}] 模型#2 无有效输出，跳过。")
            continue

        # ---------- ✅ 新的 JSON 解析 + _cluster_id 部分 ----------
        try:
            # 1) 先用修复解析器，把模型#2输出变成 list[dict]
            cluster_json_list = parse_model2_output_to_json_list(
                output_cluster,
                batch_idx=b+1,
            )

            if not cluster_json_list:
                tqdm.write(f"[批次 {b+1}] ⚠ 本批次未解析出任何合法 JSON 对象，跳过。")
                continue

            # 2) 从该批次中推断日期（你自己的逻辑）
            date_str = infer_date_for_batch(cluster_json_list, batch_lines)
            batch_id = f"B{b+1}"

            # 3) 分配全局 _cluster_id：日期 + 批次号 + 序号 ...
            cluster_json_list = assign_global_cluster_ids(
                cluster_json_list,
                date_str,
                batch_id,
            )

            # 4) 再转成 jsonl 文本（供后续 aggregate_cluster_outputs 使用）
            output_cluster_with_ids = "\n".join(
                json.dumps(c, ensure_ascii=False) for c in cluster_json_list
            )
            batch_cluster_outputs.append(output_cluster_with_ids)

        except Exception as e:
            tqdm.write(f"[批次 {b+1}] ⚠ 添加 _cluster_id 失败：{e}")
            continue
        # ---------- JSON 解析部分结束 ----------

    except Exception as e:
        tqdm.write(f"[批次 {b+1}] ❌ 出错：{e}")
        continue

    time.sleep(SLEEP_BETWEEN)
            

# ====================================================
# 模型 #3：日话题簇聚合
# ====================================================

# --- 聚合当天所有批次的话题簇 jsonl ---
all_cluster = aggregate_cluster_outputs(batch_cluster_outputs)

# --- 模型 #3：日话题簇聚合 ---
user_prompt3 = build_user_prompt_cluster_agg(all_cluster)
output_cluster_agg = call_ark_chat_completions(
    api_url=API_URL,
    api_key=API_KEY,
    model=V3_1_MODEL_ID,           # 智能体3用的模型
    system_prompt=system_prompt03,  # 日话题簇聚合提示词
    user_prompt=user_prompt3,
    temperature=TEMPERATURE,
    max_tokens=MAX_TOKENS,
    timeout=TIMEOUT_SEC,
    retries=RETRIES,
)

# 子话题簇（模型#2 输出，经 aggregate 后的 all_cluster，本身就是 jsonl）
parsed_subclusters = [
    json.loads(line.strip())
    for line in all_cluster.strip().splitlines()
    if line.strip()
]

# 聚合簇（模型#3 输出，如果不太干净，可以用 parse_jsonl_text 包一下）
parsed_clusters = normalize_model3_clusters(output_cluster_agg, parsed_subclusters)

# ====================================================
# 计算 Top5 & 写入版本 jsonl
# ====================================================

top5_results = extract_top5_heat_clusters(parsed_clusters, jsonl_lines01, top_k=5)
final_result = attach_discussion_points(top5_results, parsed_subclusters)

# 输出检查（可暂时打开）
# for row in final_result:
#     print(json.dumps(row, ensure_ascii=False, indent=2))

append_daily_top5_to_version_jsonl(
    final_result,
    version_jsonl_path=VERSION_TOP5_JSONL,  # 这个路径你可以按版本号动态改
)



准备处理 1201 条，共 5 批（每批 300 条）。


🔥 批处理进度:   0%|                                                                                                                                                                                         | 0/5 [00:00<?, ?批/s]

In [5]:
from tqdm import tqdm
import time
import json
from pathlib import Path
system_prompt04 = load_system_prompt(PROMPT_MD_PATH04)  # 版本聚合（模型4）# ====================================================
# （可选）模型 #4：版本聚合
# ====================================================
DAILY_TOP5_PATH = Path(VERSION_TOP5_JSONL)          # 你现有的多日 top5 文件路径，比如 "daily_TOP5.jsonl"
VERSION_AGG_INPUT_PATH = Path("version_agg_input.jsonl")  # 抽取后的输入文件（给模型#4）

# 1) 读取多日 daily_TOP5
if not DAILY_TOP5_PATH.exists():
    raise FileNotFoundError(f"找不到 daily_TOP5 文件：{DAILY_TOP5_PATH}")

daily_top5 = read_jsonl_file(DAILY_TOP5_PATH)
if not daily_top5:
    raise ValueError("daily_TOP5 文件为空或无可解析 JSON 行，无法做版本聚合。")

# 2) 抽取最小输入 -> 写入 version_agg_input.jsonl
agg_top5_text = build_version_agg_input_jsonl_text(daily_top5, max_points_per_row=3)

if not agg_top5_text.strip():
    raise ValueError("抽取后 agg_top5_text 为空：请检查 daily_TOP5 每行是否都有 _daily_top_id/日期/聚合话题簇/讨论点。")

VERSION_AGG_INPUT_PATH.parent.mkdir(parents=True, exist_ok=True)
VERSION_AGG_INPUT_PATH.write_text(agg_top5_text + "\n", encoding="utf-8")
version_agg_input_text = agg_top5_text
# 3) 喂给模型#4

user_prompt4 = build_user_prompt_version_agg(version_agg_input_text)

output_version_agg = call_ark_chat_completions(
    api_url=API_URL,
    api_key=API_KEY,
    model=V3_1_MODEL_ID,
    system_prompt=system_prompt04,
    user_prompt=user_prompt4,
    temperature=TEMPERATURE,
    max_tokens=MAX_TOKENS,
    timeout=TIMEOUT_SEC,
    retries=RETRIES,
)

version_clusters: list[dict] = []   # ✅ 用内置类型，运行时不会 NameError
for i, line in enumerate(output_version_agg.strip().splitlines(), start=1):
    s = line.strip()
    if not s:
        continue
    try:
        obj = json.loads(s)
    except Exception as e:
        print(f"[模型#4版本聚合输出] 第{i}行 JSON解析失败：{e}\n原文：{s[:200]}...")
        continue
    version_clusters.append(obj)

print(version_clusters)
# ✅ 直接用前面读过的 daily_top5
#version_top5 = compute_version_heat_topk(
    #version_clusters,
    #daily_top5,   # 这里用的是内存里的 daily_top5
    #top_k=5,
#)
#print(version_top5)

[{'话题簇': '测试版本下载安装咨询', '讨论点': ['地球测试版本下载链接发放时间与安装方式的具体询问', '玩家询问安装包发放时间（下午）、下载平台（安卓/iOS/模拟器）及下载方式', '游戏客户端安装包下载、文件大小及安装问题'], '日期id列表': [{'日期': '2025-12-03', 'id': '2025-12-03_T01'}]}, {'话题簇': '地球游戏核心玩法咨询', '讨论点': ['地球游戏核心玩法机制咨询，包括走格子占地、挖矿抓矿工等具体玩法细节', '玩家对游戏核心玩法类型（地块走格子策略、挂机放置、画风风格）的询问与讨论', '游戏核心玩法（探索占格子、打怪、收集装备）的理解与咨询'], '日期id列表': [{'日期': '2025-12-02', 'id': '2025-12-02_T02'}]}, {'话题簇': '殖装系统机制与策略', '讨论点': ['殖装系统解锁条件与使用限制', '殖装系统的角色配置规则，包括专属金色天赋、殖装颜色选择限制及234位置自选机制', '殖装系统开局固定殖装与后续殖装选择更换机制', '一周目/二周目石板颜色选择与初始殖装颜色的匹配策略', '殖装格子数量不足对套装适配的影响', '殖装系统的升级方法与途径', '殖装获取、升级、星级机制及大袋熊等具体殖装使用策略', '二星殖装解锁条件与阵容搭配策略', '殖装系统的基础机制与操作理解'], '日期id列表': [{'日期': '2025-12-03', 'id': '2025-12-03_T05'}, {'日期': '2025-12-04', 'id': '2025-12-04_T01'}, {'日期': '2025-12-05', 'id': '2025-12-05_T01'}]}, {'话题簇': '精神值管理系统', '讨论点': ['角色精神状态管理系统，包括精神值归零后果、异食癖效果、恐惧症debuff以及驾驶员作为消耗品的机制', '精神污染状态的特殊效果，讨论30点精神以下时伤害翻倍的机制优势', '理智值恢复方式与道具回复效果探讨', '精神力恢复方式、驾驶员死亡与复活机制、物品保留规则', '低精神力状态下的战力增强效果', '精神值回复道具（如烟）的效果与驾驶员消耗品特性', '低精神值状态的特殊效果

In [10]:
path = Path(VERSION_TOP5_JSONL)
print("✅ writing to:", path.resolve())
print("✅ cwd:", Path.cwd().resolve())


✅ writing to: E:\项目\玩家社群分析智能体\玩家发言整理（供运营侧）\玩家发言总结_版本总结(多天)\test_top5.jsonl
✅ cwd: E:\项目\玩家社群分析智能体\玩家发言整理（供运营侧）\玩家发言总结_版本总结(多天)


In [14]:
daily_top5 = read_jsonl_file(DAILY_TOP5_PATH)

def pick_id(r):
    return (r.get("_daily_top_id") or r.get("id") or "").strip()

ids_raw  = [pick_id(r) for r in daily_top5_raw]
ids_read = [pick_id(r) for r in daily_top5]

print("len(raw) =", len(daily_top5_raw), "len(read) =", len(daily_top5))
print("head raw ids  =", ids_raw[:10])
print("head read ids =", ids_read[:10])

missing = sorted(set(ids_raw) - set(ids_read))
extra   = sorted(set(ids_read) - set(ids_raw))
print("missing ids count =", len(missing), "extra ids count =", len(extra))
print("missing ids head =", missing[:20])


len(raw) = 20 len(read) = 20
head raw ids  = ['2025-12-03_T01', '2025-12-02_T02', '2025-12-03_T03', '2025-12-03_T04', '2025-12-03_T05', '2025-12-04_T01', '2025-12-04_T02', '2025-12-04_T03', '2025-12-04_T04', '2025-12-04_T05']
head read ids = ['2025-12-03_T01', '2025-12-02_T02', '2025-12-03_T03', '2025-12-03_T04', '2025-12-03_T05', '2025-12-04_T01', '2025-12-04_T02', '2025-12-04_T03', '2025-12-04_T04', '2025-12-04_T05']
missing ids count = 0 extra ids count = 0
missing ids head = []


## 开始运行(不可动）

In [ ]:
from tqdm import tqdm
import time
import json
from pathlib import Path

# ... 前置：系统提示、create_intent_excel_styled(EXCEL_FILE)、jsonl_lines01 等

system_prompt01 = load_system_prompt(PROMPT_MD_PATH01)  # 筛相关
system_prompt02 = load_system_prompt(PROMPT_MD_PATH02)  # 做话提簇
system_prompt03 = load_system_prompt(PROMPT_MD_PATH03)  # 话题簇聚合 / 校正（智能体4）
system_prompt04 = load_system_prompt(PROMPT_MD_PATH04)  # 话提簇玩家观点感受分析


sub_opinion_map = {}
# 用于存放当天所有批次的话题簇 JSON 行（给智能体4用）
batch_cluster_outputs  = []
total = len(jsonl_lines01)
if total == 0:
    print("没有可处理的数据。")
else:
    total_batches = (total + BATCH_SIZE - 1) // BATCH_SIZE
    print(f"准备处理 {total} 条，共 {total_batches} 批（每批 {BATCH_SIZE} 条）。")

written_total = 0

for b in tqdm(range(total_batches), desc="🔥 批处理进度", unit="批"):
    start = b * BATCH_SIZE
    end = min(start + BATCH_SIZE, total)
    batch_lines = jsonl_lines01[start:end]

    try:
        # --- 模型 #1：筛相关 ---
        user_prompt1 = build_user_prompt_filter(batch_lines)
        output_filter = call_ark_chat_completions(
            api_url=API_URL,
            api_key=API_KEY,
            model=V3_MODEL_ID,
            system_prompt=system_prompt01,
            user_prompt=user_prompt1,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            timeout=TIMEOUT_SEC,
            retries=RETRIES,
        )
        
        if not output_filter:
            tqdm.write(f"[批次 {b+1}] 模型#1 无有效输出，跳过。")
            continue
        filter_count = sum(1 for line in output_filter.splitlines() if line.strip())
        tqdm.write(f"[批次 {b+1}] 模型#1 筛后保留 {filter_count} 条")
        written_total += filter_count   # 如果不需要总数，可以删掉这一行
        
        # --- 模型 #2：话题簇划分 ---
        user_prompt2 = build_user_prompt_clsuter(output_filter)
        output_cluster = call_ark_chat_completions(
            api_url=API_URL,
            api_key=API_KEY,
            model=V3_1_MODEL_ID,
            system_prompt=system_prompt02,
            user_prompt=user_prompt2,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            timeout=TIMEOUT_SEC,
            retries=RETRIES,
        )
        
        

        if not output_cluster:
            tqdm.write(f"[批次 {b+1}] 模型#2 无有效输出，跳过。")
            continue
        try:
            cluster_json_list = []
            for line in output_cluster.strip().splitlines():
                line = line.strip()
                if not line.startswith("{"):
                    continue
                obj = json.loads(line)
                # 将 "话题簇1" → "话题簇"，便于后续统一处理
                for key in list(obj.keys()):
                    if key.startswith("话题簇") and key != "话题簇":
                        obj["话题簇"] = obj.pop(key)
                cluster_json_list.append(obj)

            # 假设从输出中拿日期
            date_str = infer_date_for_batch(cluster_json_list, batch_lines)  # 你可以从上下文获得更准确值
            batch_id = f"B{b+1}"
            cluster_json_list = assign_global_cluster_ids(cluster_json_list, date_str, batch_id)
        
            output_cluster_with_ids = "\n".join(json.dumps(c, ensure_ascii=False) for c in cluster_json_list)
            batch_cluster_outputs.append(output_cluster_with_ids)
        except Exception as e:
            tqdm.write(f"[批次 {b+1}] ⚠ 添加 _cluster_id 失败：{e}")
            continue

       
       
    except Exception as e:
        tqdm.write(f"[批次 {b+1}] ❌ 出错：{e}")
        continue

    time.sleep(SLEEP_BETWEEN)



# --- 每日话提簇聚合成jsonl ---
all_cluster = aggregate_cluster_outputs(batch_cluster_outputs)

# --- 模型 #3：日话提簇聚合 ---
user_prompt3 = build_user_prompt_cluster_agg(all_cluster)
output_cluster_agg = call_ark_chat_completions(
    api_url=API_URL,
    api_key=API_KEY,
    model=V3_1_MODEL_ID,           # 或你给智能体4选的模型
    system_prompt=system_prompt03,  # 话题簇聚合/校正提示词
    user_prompt=user_prompt3,
    temperature=TEMPERATURE,
    max_tokens=MAX_TOKENS,
    timeout=TIMEOUT_SEC,
    retries=RETRIES,
)

#每日总发言转单条jonsl格式
parsed_subclusters = [
    json.loads(line.strip()) for line in all_cluster.strip().splitlines() if line.strip()
]

# 解析原始发言 jsonl，为按时间轴筛选对话用
parsed_msgs = [json.loads(line.strip()) for line in jsonl_lines01 if line.strip()]

#每日聚合后话提簇转单条jsonl格式
parsed_clusters = [json.loads(line) for line in output_cluster_agg.strip().splitlines() if line.strip()]

#计算每日讨论热度top5
top5_results = extract_top5_heat_clusters(parsed_clusters, jsonl_lines01, top_k=5)

#######提取子话提簇讨论点###########################################################
####### 建子簇索引：_cluster_id -> 子簇对象 ########
sub_map = {}
for row in parsed_subclusters:
    cid = row.get("_cluster_id")
    if isinstance(cid, str) and cid:
        sub_map[cid] = row

####### 在 Top5 循环里，对每个【子话题簇】单独跑模型#4 ########
for cluster in top5_results:
    date = cluster["日期"]
    # 聚合簇的整体时间轴（只用来兜底）
    cluster_time_axis = cluster["时间轴"]
    cid_list = cluster.get("子话题簇列表", [])

    for cid in cid_list:
        sub = sub_map.get(cid)
        if not sub:
            print(f"\n[聚合话题簇：{cluster.get('聚合话题簇')}] 子话题簇 ID：{cid}")
            print("  ⚠ 在 sub_map / parsed_subclusters 中未找到该 _cluster_id。")
            continue

        
        # ---- 1）从子话题簇【标题】里提取时间轴 ----
        sub_title = sub.get("话题簇", "") or ""
        sub_time_axis = extract_time_axis_from_title(sub_title)

        # ---- 2）按【子话提簇时间轴】从原始发言里匹配 dialogs ----
        dialogs = match_dialogs_by_time(parsed_msgs, date, sub_time_axis)

        # ---- 3）构造 discussion_point（不打印，只用来写 prompt）----
        discussion_point = (
    
            sub.get("核心对象/机制")
            or sub.get("话题簇")
            or ""
        )
        topic_id = sub.get("_cluster_id", cid)
        
        # ---- 4）模型4 玩家发言感受分析总结----
        user_prompt4 = build_user_prompt_subcluster_opinion(
            topic_id=topic_id,
            discussion_point=discussion_point,
            dialogs=dialogs,
        )
        if not isinstance(user_prompt4, str) or not user_prompt4.strip():
            print(f"  ⚠ user_prompt4 为空或非法，跳过本子话题簇：{cid}")
            continue
        opinion_output = call_ark_chat_completions(
            api_url=API_URL,
            api_key=API_KEY,
            model=V3_1_MODEL_ID,             
            system_prompt=system_prompt04,   
            user_prompt=user_prompt4,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            timeout=TIMEOUT_SEC,
            retries=RETRIES,
        )


       
    opinion_obj = parse_and_normalize_opinion_output(
        opinion_output=opinion_output,
        topic_id=topic_id,
        discussion_point=discussion_point,
    )
    sub_opinion_map[topic_id] = opinion_obj
# ===================================
# 组装 Top5 + 观点总结，并写入 jsonl
# ===================================



准备处理 2308 条，共 8 批（每批 300 条）。


🔥 批处理进度:   0%|                                                                                                                                                       | 0/8 [06:33<?, ?批/s]      

[批次 1] 模型#1 筛后保留 240 条


🔥 批处理进度:  12%|█████████████████▊                                                                                                                            | 1/8 [12:29<48:22, 414.62s/批]      

[批次 2] 模型#1 筛后保留 206 条


🔥 批处理进度:  25%|███████████████████████████████████▌                                                                                                          | 2/8 [20:44<38:45, 387.59s/批]      

[批次 3] 模型#1 筛后保留 283 条


🔥 批处理进度:  38%|█████████████████████████████████████████████████████▎                                                                                        | 3/8 [21:22<36:31, 438.36s/批]

In [9]:
final_top5_with_opinion = build_daily_top5_opinion_records(
    top5_results=top5_results,
    sub_opinion_map=sub_opinion_map,
)

append_daily_top5_to_version_jsonl(
    final_result=final_top5_with_opinion,
    version_jsonl_path="version_daily_top5_with_opinion.jsonl",
)

✅ 已将当日 Top5（含 _idx 和 _daily_top_id）追加写入: version_daily_top5_with_opinion.jsonl


In [23]:
print(cid_list)
print(sub)

print(sub_title)
print(sub_time_axis)
print(discussion_point)
print(dialogs)
print(cid)

['2025-12-02_B1_03', '2025-12-02_B1_14', '2025-12-02_B2_07']
{'话题簇': '游戏物品异常问题反馈（2025-12-02 22:57:27-22:57:31）', '核心对象/机制': '游戏内物品异常消失问题的反馈与处理', '_cluster_id': '2025-12-02_B2_07'}
游戏物品异常问题反馈（2025-12-02 22:57:27-22:57:31）
22:57:27-22:57:31
游戏内物品异常消失问题的反馈与处理
[{'发言日期': '2025-12-02', '发言时间': '22:57:27', '玩家ID': '无为无畏(2514177080)', '玩家消息': '[图片]'}, {'发言日期': '2025-12-02', '发言时间': '22:57:31', '玩家ID': '无为无畏(2514177080)', '玩家消息': '东西没了'}]
2025-12-02_B2_07


In [6]:
print(opinion_output)

NameError: name 'opinion_output' is not defined

In [14]:
print(sub_time_axis)